# Altair Choropleths play

Here I load in James' presidential polls dataset from the teaching notebook in this dir and attempt some rudimentary WAYS ideas.

In [ ]:
%%capture
# capture suppresses output of the below:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import geopandas as gpd 
import pandas as pd
import altair as alt

geo_states = gpd.read_file('choropleth_teaching/gz_2010_us_040_00_500k.json')
df_polls = pd.read_csv('choropleth_teaching/presidential_poll_averages_2020.csv')

In [ ]:
df_polls.head()

On how many dates was the polling done?

In [ ]:
len(df_polls.modeldate.unique())

Filter our poll data to remove third party candidates.

In [ ]:
df_polls = df_polls[
    (df_polls.candidate_name == 'Donald Trump') |
    (df_polls.candidate_name == 'Joseph R. Biden Jr.')
]

The geo_states variable has polygons for each state.

In [ ]:
geo_states.head()

In [ ]:
trump_data = df_polls[
    df_polls.candidate_name == 'Donald Trump'
]

biden_data = df_polls[
    df_polls.candidate_name == 'Joseph R. Biden Jr.'
]

Our spatial and poll data have the name of the state in common. We will change the name of the state to NAME to match our geospatial dataframe.

In [ ]:
trump_data.columns = ['cycle', 'NAME', 'modeldate', 'candidate_name', 'pct_estimate', 'pct_trend_adjusted']
biden_data.columns = ['cycle', 'NAME', 'modeldate', 'candidate_name', 'pct_estimate', 'pct_trend_adjusted']

We can join the geospatial and poll data using the NAME column (the name of the state).

In [ ]:
# Create separate date frame for trump and biden
# Add the poll data
geo_states_trump = geo_states.merge(trump_data, on='NAME')
geo_states_biden = geo_states.merge(biden_data, on='NAME')

In [ ]:
(geo_states_trump.modeldate == '11/03/2020').value_counts()

In [ ]:
geo_states_biden.head()

### Create choropleth plot function

In [ ]:
def usa_choro(candidate_geo_states, color, title):
    """Plot a choropleth of the US states with the candidate vote percentage mapped to the color;
    pass in altair bin and scale objects"""
    chart = alt.Chart(candidate_geo_states, title=title).mark_geoshape()
    chart = chart.encode(
        color,
        tooltip=['NAME', 'pct_estimate']
    ).properties(
        width=500,
        height=300
    ).project(
        type='albersUsa'
    )
    return chart

### Import WAYS "metavisualisation" histogram

In [ ]:
import sys; sys.path.insert(0, '..')
from ways_py.ways import Ways

## Adding interactivity with Jupyter interact widgets

Here I attempt to offer the user working on the the US presidential poll choropleth visualisation some options related to the colour-binning.

In [ ]:
from ipywidgets import interact, widgets

In [ ]:
# Get an ordered list of the dates (as strings) on which polling occured
import datetime
unsorted_datestrings = list(set(list(geo_states_biden['modeldate'])))
dates = sorted(unsorted_datestrings, key=lambda x: datetime.datetime.strptime(x, '%m/%d/%Y'))

Get a list of possible altair scales from [here](https://altair-viz.github.io/user_guide/generated/core/altair.ScaleType.html#altair.ScaleType) and create a list:

In [ ]:
scales = ['linear', 'log', 'pow', 'sqrt', 'symlog', 'identity', 'sequential', 'time', 'utc', 'quantile', 'quantize', 'threshold', 'bin-ordinal', 'ordinal', 'point', 'band']

In [ ]:
# Create widgets
extent_slider = widgets.IntRangeSlider(value=[0,100], min=0, max=100)
date_dropdown = widgets.SelectionSlider(value='11/03/2020', options=dates)
scales_dropdown = widgets.Dropdown(value='band', options=scales)

# Define interactive plot function
@interact
def interact_usa_choro(candidate=['Trump', 'Biden'], date=date_dropdown, bin=True, maxbins=(2, 100, 1), scale=scales_dropdown, extent=extent_slider):
    
    # Choose the candidate dataset
    candidate_geo_states = geo_states_trump if candidate == 'Trump' else geo_states_biden
    
    # Choose which polling date to display
    candidate_geo_states = candidate_geo_states[
        (candidate_geo_states.modeldate == date)
    ]
    
    # Give the choropleth plot a title
    title = 'Poll estimate for ' + candidate + ' on ' + date
    
    # Create the bin, scale and color objects
    if bin: # if bin is False, leave as bool
        bin = alt.Bin(maxbins=maxbins, extent=extent, step=10)
    column = "pct_estimate"
    # TODO: add default color scheme
    color = alt.Color(column, bin=bin, scale=alt.Scale(type=scale))
    
    # Generate both plots and place them horizontally
    return Ways.altair_meta_hist(usa_choro(candidate_geo_states, color, title), column)

Notes:
1. Setting the scale to `log` doesn't work unless `bin` is **unchecked**
2. Because the `extent` data range plugs into the `alt.Bin` object, this will only work if `bin` is **checked**
3. There is no way to pre-populate dropdowns with all available options for a given function argument